# Spacy/Medspacy IAA

## Resources

Prodigy forum answer about IAA for spans https://support.prodi.gy/t/proper-way-to-calculate-inter-annotator-agreement-for-spans-ner/5760

Spacy scorer object https://spacy.io/api/scorer

## End Goal

### Functionality

Provide a collection of methods to evaluate IAA between _n_ arbitrary spacy `doc` objects. Provide methods that aid in error analysis such as providing lists of differences.

Priorities:
* Pairwise F1
    * configurable strict/loose matching
    * configurable inclusion of labels/attributes (calculate just span vs span+class agreement)

* Imported python files
    * reasonable docstrings on methods/classes
    
* Unit tests
    * add CI to repo for automated testing later

Extra features:
* List of differences between docs
* 

Expected challenges
* Spacy scorer functions are useful, but _only_ do strict span matching
* Fewer resources (obviously?) available for comparisons between 3+ docs


# Testing Section

In [ ]:
import spacy

In [ ]:
#!python -m spacy download en_core_web_md

In [ ]:
nlp = spacy.load("en_core_web_sm")
nlp2 = spacy.load("en_core_web_md")

In [ ]:
doc = nlp("this is a test document made in utah or mississippi, or salt lake city.")

In [ ]:
doc.ents

In [ ]:
doc2 = nlp2("this is a test document made in utah or mississippi, or salt lake city.")

In [ ]:
doc2.ents

In [ ]:
from spacy.tokens import Span
spand = list()
spand += [Span(doc, 2, 4, label="PERSON"),Span(doc,7,8,label="GPE"),Span(doc,9,10,label="PERSON"),Span(doc,13,14,label="GPE"),Span(doc,14,15,label="GPE")]

# Add the span to the doc's entities
doc.ents = spand

# Print entities' text and labels
print([(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
#Run below cells before this
tp,fp,fn = agreement(doc,doc2,1,1)
print(tp,fp,fn)
print(pairwise_f1(tp,fp,fn))

In [2]:
import spacy

In [3]:
nlp = spacy.load("en_core_web_sm")

test_str = "This is a test document. For testing lol."
test_str_2 = "This is a test document. For testing lol."

doc = nlp(test_str)
doc2 = nlp(test_str)

from spacy.tokens import SpanGroup

spans = [doc[0:1], doc[1:3]]
group = SpanGroup(doc, name="errors", spans=spans, attrs={"annotator": "matt"})
doc.spans["errors"] = group
group = SpanGroup(doc, name="entity1", spans=spans, attrs={"annotator": "John"})
#doc.spans["entity"] = group


spans = [doc2[0:1], doc2[1:5],doc2[4:5]]
group = SpanGroup(doc2, name="errors", spans=spans, attrs={"annotator": "matt"})
doc2.spans["errors"] = group
group = SpanGroup(doc2, name="entity1", spans=spans, attrs={"annotator": "John"})
#doc2.spans["entity"] = group

In [8]:
IAA.corpus_agreement([doc.spans['errors']],doc.ents,labels=0)

Input Error: Input must be iterable of spacy documents, or dataframe.


In [5]:
from spacy.tokens import Span
import spacy

nlp = spacy.load("en_core_web_sm")

doc = nlp("this is a test document made in utah or mississippi, or salt lake city.")

spand = list()
spand += [Span(doc, 2, 4, label="PERSON"),Span(doc,7,8,label="GPE"),Span(doc,9,10,label="PERSON"),Span(doc,13,14,label="GPE"),Span(doc,14,15,label="GPE")]

# Add the span to the doc's entities
doc.ents = spand

print([(ent.text, ent.label_) for ent in doc.ents])

from spacy.tokens import SpanGroup

spans = [doc[7:8], doc[8:10]]
group = SpanGroup(doc, name="errors", spans=spans, attrs={"annotator": "matt"})
doc.spans["errors"] = group
print(doc.spans['errors'])
print(doc.spans['errors'][0].label_)

[('a test', 'PERSON'), ('utah', 'GPE'), ('mississippi', 'PERSON'), ('lake', 'GPE'), ('city', 'GPE')]
[utah, or mississippi]



In [ ]:
print(doc.spans['errors'])
for span in doc.spans['errors']:
    print('what')

In [ ]:
agreement(doc.ents,doc.spans['errors'],labels=0)

In [ ]:
agreement(doc,doc2,ent_or_span='ent')

In [ ]:
type(doc.ents[0]) is spacy.tokens.span.Span
type(doc.spans['errors']) is spacy.tokens.span_group.SpanGroup

type(doc)

# Code

In [ ]:
from quicksectx import IntervalNode, IntervalTree, Interval #note that you need the quicksectx library

#In order to make the code a little more adaptable for situations of multiple overlapping entities, as well as for 
#transparency and testing the code, I wrote the overlaps code to output a mapping of which entities are being matched. 
#Then agreement can parse this output for how many valid overlaps exist.

#This makes the code a little more complicated to understand, but I think it makes everything more transparent and adaptable.

def overlaps(doc1_ents, doc2_ents,labels=1):
    '''Calculates overlapping entities between two spacy documents. Also checks for matching labels if label=1.
    
    Return:
        Dictionaries with the mapping of matching entity indices:
            keys: entity index from one annotation
            value: matched entity index from other annotation
        
        Ex: "{1 : [2] , 3 : [4,5]}" means that entity 1 from doc1 matches entity 1 in doc2, and entity 3 in doc1 matches 
        entity 4 and 5 from doc2.
    '''
    
    doc1_matches = dict()
    doc2_matches = dict()
    
    tree = IntervalTree()
    for index2,ent2 in enumerate(doc2_ents):
        tree.add(ent2.start_char,ent2.end_char,index2)
    
    for index1,ent1 in enumerate(doc1_ents):
        matches = tree.search(ent1.start_char,ent1.end_char)
        for match in matches:
            index2 = match.data #match.data is the index of doc2_ents
            if ((labels == 0) | (doc2_ents[index2].label_ == ent1.label_)):
                if index1 not in doc1_matches.keys():
                    doc1_matches[index1] = [index2]
                else:
                    doc1_matches[index1].append(index2)
                if index2 not in doc2_matches.keys():
                    doc2_matches[index2] = [index1]
                else:
                    doc2_matches[index2].append(index1)
                
    return doc1_matches, doc2_matches

In [ ]:
### This is the old, less efficient code. The newer code uses a tree search instead of the nested for-loop.

def old_overlaps(doc1_ents, doc2_ents,labels):
    '''Old code for calculating overlapping entities between two spacy documents. Also checks for matching labels if label=1.
    
    Return:
        Dictionaries with the mapping of matching entity indices:
            keys: entity index from one annotation
            value: matched entity index from other annotation
        
        Ex: "{1 : [2] , 3 : [4,5]}" means that entity 1 from doc1 matches entity 1 in doc2, and entity 3 in doc1 matches 
        entity 4 and 5 from doc2.
    '''
    
    doc1_matches = dict()
    doc2_matches = dict()

    for index1,ent1 in enumerate(doc1_ents):
        for index2,ent2 in enumerate(doc2_ents):
            if (ent1.end_char >= ent2.start_char) & (ent1.start_char <= ent2.end_char) & ((labels==0) | (ent1.label_ == ent2.label_)):
                if index1 not in doc1_matches.keys():
                    doc1_matches[index1] = [index2]
                else:
                    doc1_matches[index1].append(index2)
                if index2 not in doc2_matches.keys():
                    doc2_matches[index2] = [index1]
                else:
                    doc2_matches[index2].append(index1)
                
    return doc1_matches, doc2_matches
    

In [ ]:
def exact_match(doc1_ents, doc2_ents, labels):
    '''calculate whether two ents have exact overlap
    returns bool
    '''
    
    doc1_matches = dict()
    doc2_matches = dict()

    doc1_ent_dict = dict()
    doc2_ent_dict = dict()
    
    for index1,ent1 in enumerate(doc1_ents):
        if labels == 1: #If checking for labels, then include this in the tuple's to-be-compared elements
            doc1_ent_dict[(ent1.start_char,ent1.end_char,ent1.label_)] = index1
        else:
            doc1_ent_dict[(ent1.start_char,ent1.end_char)] = index1
            
    for index2,ent2 in enumerate(doc2_ents):
        if labels == 1:    
            doc2_ent_dict[(ent2.start_char,ent2.end_char,ent2.label_)] = index2
        else:
            doc2_ent_dict[(ent2.start_char,ent2.end_char)] = index2
        
    doc1_ent_set = set(doc1_ent_dict.keys())
    doc2_ent_set = set(doc2_ent_dict.keys())
    
    matched_ents = doc1_ent_set.intersection(doc2_ent_set)
    
    for match in matched_ents:
        index1 = doc1_ent_dict[match]
        index2 = doc2_ent_dict[match]
        doc1_matches[index1] = [index2]
        doc2_matches[index2] = [index1]
        
    return doc1_matches, doc2_matches
    

In [ ]:
def agreement(doc1, doc2, loose=1, labels=1, ent_or_span = 'ent'):
    '''Calculates confusion matrix for agreement between two documents.
    
       returns true positive, false positive, and false negative
    '''
    if (type(doc1) is tuple) or (type(doc1) is spacy.tokens.span_group.SpanGroup) and \
    (type(doc2) is tuple) or (type(doc2) is spacy.tokens.span_group.SpanGroup):
        doc1_ents = doc1
        doc2_ents = doc2
    elif (type(doc1) is spacy.tokens.doc.Doc) and (type(doc2) is spacy.tokens.doc.Doc):
        if ent_or_span == 'ent':
            doc1_ents = doc1.ents
            doc2_ents = doc2.ents
        elif ent_or_span == 'span':
            if len(doc1.spans) > 1:
                #raise error
                print("Error: cannot distinquish which span group to use from doc1.")
                return
            else:
                span_group = list(doc1.spans.keys())[0]
                doc1_ents = doc1.spans[span_group]
                doc2_ents = doc2.spans[span_group]
        else:
            #raise error
            print("Error: Must select 'span' or 'ent' for ent_or_span option.")
            return
    else:
        #raise error
        print("Error: Input must be of type 'tuples', 'spacy.tokens.span_group.SpanGroup', or 'spacy.tokens.doc.Doc'")
        return
        
    if loose:
        doc1_matches, doc2_matches = overlaps(doc1_ents, doc2_ents, labels)
    else:
        doc1_matches, doc2_matches = exact_match(doc1_ents, doc2_ents, labels)
    
    return conf_matrix(doc1_matches,doc2_matches,len(doc1_ents),len(doc2_ents))


In [ ]:
def conf_matrix(doc1_matches,doc2_matches,doc1_ent_num,doc2_ent_num):

    doc1_match_num = len(doc1_matches.keys())
    doc2_match_num = len(doc2_matches.keys())
    
    duplicate_matches = 0
    for value in doc2_matches.values():
        duplicate_matches += len(value) - 1
    
    tp = doc1_match_num - duplicate_matches #How many entity indices from doc1 matched, minus duplicated matches
    fp = doc2_ent_num - doc2_match_num #How many entities from doc2 that didn't match
    fn = doc1_ent_num - doc1_match_num #How many entities from doc1 that didn't match
    
    return (tp,fp,fn)

In [ ]:
def pairwise_f1(tp,fp,fn):
    '''calculate f1 given true positive, false positive, and false negative values'''
    
    return (2*tp)/float(2*tp+fp+fn)

In [ ]:
def corpus_agreement(docs1, docs2, loose=1, labels=1,ent_or_span='ent'):
    '''calculate f1 over an entire corpus of documents'''
    corpus_tp, corpus_fp, corpus_fn = (0,0,0)
    
    if type(docs1[0]) is spacy.tokens.doc.Doc:
        for i, doc1 in enumerate(docs1):
            tp,fp,fn = agreement(doc1, docs2[i],loose,labels,ent_or_span)
            corpus_tp += tp
            corpus_fp += fp
            corpus_fn += fn
    elif type(docs1) is pandas.core.frame.DataFrame:
        for doc_name in docs1['doc name'].unique():
            docs1_df = docs1[docs1['doc name'] == doc_name]
            docs2_df = docs2[docs2['doc name'] == doc_name]
            doc1_matches,doc2_matches = df_overlaps(docs1_df,docs2_df)
            tp,fp,fn = conf_matrix(doc1_matches,doc2_matches,docs1_df.shape[0],docs2_df.shape[0])
            corpus_tp += tp
            corpus_fp += fp
            corpus_fn += fn
    else:
        #raise error
        print('Input Error: Input must be iterable of spacy documents, or dataframe.')
        return
    
    data = {'IAA' : [pairwise_f1(corpus_tp,corpus_fp,corpus_fn)], 'Recall' : [tp/float(tp+fp)], 'Precision' : [tp/float(tp+fn)],\
           'True Positives' : [tp] , 'False Positives' : [fp], 'False Negative' : [fn]}
    
    return pd.DataFrame(data)

# Tutorial

In this tutorial I will go over the basic, front-end usage of calculating IAA between 2 annotators.

In [ ]:
import spacy
import medspacy
nlp1 = spacy.load("en_core_web_sm")
nlp2 = spacy.load("en_core_web_md")
#!python -m spacy download en_core_web_sm
#!python -m spacy download en_core_web_md

#Note for John: Get better examples or make my own entities
doc1 = nlp1("this is a test document made in utah or mississippi, or salt lake city.")
doc2 = nlp2("this is a test document made in utah or mississippi, or salt lake city.")

print('doc1.ents: ',doc1.ents)
print('doc2.ents: ',doc2.ents)

Above we made two documents using spacy's NER packages. Document 2 added more entities than document 1. Let's calculate the IAA between these documents!

In [ ]:
corpus_agreement([doc1],[doc2])

'corpus_agreement' calculates the agreement between two lists of documents. Note the brackets around 'doc1' and 'doc2' so they are passed in as lists of size 1 each.

'corpus_agreement' can also take options to be more flexible with other IAA methods. Below are the arguments:

### corpus_agreement(docs1, docs2, loose=1, labels=1,ent_or_span='ent')

docs1: list of spacy documents

docs2: list of spacy documents with same order as docs1

loose: Boolean for allowing loose matching. '1' indicates that any overlap between two spans/entities is counted towards IAA. '0' indicates that only exact matches will be allowed.

labels: Boolean to include labels when matching. Uses the .label_ attribute of entities/spans to access labels.

ent_or_span: string of whether spans or entities are being compared. If set to 'ent', code will iterate through doc1.ents and doc2.ents. If set to 'span', code will iterate through the spans within doc1 and doc2's first span group. 'span' only works if doc1 has one span group. This option may be extended to include doc._ .concepts option, or to allow multiple span groups to be compared.

Internally, corpus_agreement is calling the 'agreement' and 'pairwise_f1' functions on each pair of document in the lists. We can instead choose to call these functions separate

# Code in Development

In [ ]:
import pandas as pd
john_df = pd.read_pickle('./df_John.pkl')
#john_df = john_df[john_df['Concept Label'] == 'Symptom']
mengke_df = pd.read_pickle('./df_Mengke.pkl')

In [ ]:
from quicksectx import IntervalNode, IntervalTree, Interval #note that you need the quicksectx library

#In order to make the code a little more adaptable for situations of multiple overlapping entities, as well as for 
#transparency and testing the code, I wrote the overlaps code to output a mapping of which entities are being matched. 
#Then agreement can parse this output for how many valid overlaps exist.

#This makes the code a little more complicated to understand, but I think it makes everything more transparent and adaptable.

def df_overlaps(docs1_df, docs2_df,labels=1):
    '''Calculates overlapping entities between two spacy documents. Also checks for matching labels if label=1.
    
    Return:
        Dictionaries with the mapping of matching entity indices:
            keys: entity index from one annotation
            value: matched entity index from other annotation
        
        Ex: "{1 : [2] , 3 : [4,5]}" means that entity 1 from doc1 matches entity 1 in doc2, and entity 3 in doc1 matches 
        entity 4 and 5 from doc2.
    '''
    
    doc1_matches = dict()
    doc2_matches = dict()
    
    tree = IntervalTree()
    for index2,row2 in docs2_df.iterrows():
        tree.add(row2['start loc'],row2['end loc'],index2)
    
    for index1,row1 in docs1_df.iterrows():
        matches = tree.search(row1['start loc'],row1['end loc'])
        for match in matches:
            index2 = match.data #match.data is the index of doc2_ents
            if ((labels == 0) | (docs2_df.loc[index2,'Concept Label'] == row1['Concept Label'])):
                if index1 not in doc1_matches.keys():
                    doc1_matches[index1] = [index2]
                else:
                    doc1_matches[index1].append(index2)
                if index2 not in doc2_matches.keys():
                    doc2_matches[index2] = [index1]
                else:
                    doc2_matches[index2].append(index1)
                
    return doc1_matches, doc2_matches

In [ ]:
def df_corpus_agreement(docs1, docs2, loose=1, labels=1,ent_or_span='ent'):
    '''calculate f1 over an entire corpus of documents'''
    corpus_tp, corpus_fp, corpus_fn = (0,0,0)
    
    for doc_name in docs1['doc name'].unique():
        docs1_df = docs1[docs1['doc name'] == doc_name]
        docs2_df = docs2[docs2['doc name'] == doc_name]
        doc1_matches,doc2_matches = df_overlaps(docs1_df,docs2_df)
        tp,fp,fn = conf_matrix(doc1_matches,doc2_matches,docs1_df.shape[0],docs2_df.shape[0])
        corpus_tp += tp
        corpus_fp += fp
        corpus_fn += fn
    
    print('corpus tp: ',corpus_tp,'\ncorpus fp: ',corpus_fp,'\ncorpus fn: ',corpus_fn)
    
    print(tp)
    print(corpus_tp)
    
    #print("Not in doc2 annotations:\n")
    for index,row in docs1.iterrows():
        if (index not in doc2_matches.keys()):
            #print(row['Span Text'])
            break
    
    return pairwise_f1(corpus_tp,corpus_fp,corpus_fn)

In [ ]:
df_corpus_agreement(john_df,mengke_df,1)

In [ ]:
john_df_symptoms = john_df[john_df['Concept Label'] == 'Symptom']
mengke_df_symptoms = mengke_df[mengke_df['Concept Label'] == 'Symptom']

df_corpus_agreement(john_df_symptoms,mengke_df_symptoms)

In [ ]:
corpus_agreement(john_df_symptoms,mengke_df_symptoms,1)

In [ ]:
type(john_df)

In [1]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, './Integrated_code/')
import IAA_ as IAA

In [2]:
import pandas as pd
john_df = pd.read_pickle('./df_John.pkl')
#john_df = john_df[john_df['Concept Label'] == 'Symptom']
mengke_df = pd.read_pickle('./df_Mengke.pkl')

In [7]:
IAA.corpus_agreement(john_df,mengke_df,labels=0)[1]

485939.txt
366026.txt
5585.txt


IndexError: list index out of range

In [4]:
john_df['doc name'].unique()

array(['485939.txt', '366026.txt', '5585.txt', '669731.txt', '33200.txt',
       '36609.txt', '335643.txt', '52374.txt', '38580.txt', '601884.txt',
       '38757.txt', '416614.txt', '45725.txt', '56773.txt', '19070.txt',
       '628045.txt', '45509.txt', '527735.txt', '439912.txt',
       '457012.txt', '595179.txt', '575514.txt', '28056.txt',
       '655523.txt', '537108.txt', '33137.txt', '330814.txt', '18826.txt',
       '700062.txt', '26272.txt', '321062.txt', '584041.txt',
       '369557.txt', '679566.txt', '593885.txt', '379.txt', '489228.txt',
       '24881.txt', '377554.txt', '700799.txt', '26388.txt', '330335.txt',
       '18305.txt', '375911.txt', '345622.txt', '346854.txt',
       '536291.txt', '456788.txt', '5749.txt', '3271.txt'], dtype=object)

In [3]:
john_df_doc1 = john_df[john_df['doc name'] == '5585.txt']
mengke_df_doc1 = mengke_df[mengke_df['doc name'] == '5585.txt']
mengke_df_doc1

,Span Text,Concept Label,start loc,end loc,doc name
17,:,SectionHeader_HasSymptom,152,153,5585.txt
18,Left chest pain and left sided weakness,Symptom_Section,154,194,5585.txt
19,chest pain and left arm weakness. The chest pain,Symptom_Section,390,438,5585.txt
20,"denies dyspnea, visual changes, loss\nof consc...",Symptom_Section,1224,1317,5585.txt
21,Left chest pain,Symptom,155,170,5585.txt
22,left sided weakness,Symptom,175,194,5585.txt
23,chest pain,Symptom,390,400,5585.txt
24,left arm weakness,Symptom,405,422,5585.txt
25,chest pain,Symptom,428,438,5585.txt
26,radiating pain,Symptom,526,540,5585.txt


In [5]:
john_df_doc1

,Span Text,Concept Label,start loc,end loc,doc name
30,Left chest pain,Symptom,155,170,5585.txt
31,left sided weakness,Symptom,175,194,5585.txt
32,chest pain,Symptom,390,400,5585.txt
33,left arm weakness,Symptom,405,422,5585.txt
34,chest pain,Symptom,428,438,5585.txt
35,left arm weakness,Symptom,596,613,5585.txt
36,left arm weakness,Symptom,656,673,5585.txt
37,pain,Symptom,1098,1102,5585.txt
38,weakness,Symptom,1104,1112,5585.txt
39,paresthesias of the left face,Symptom,1117,1146,5585.txt


In [6]:
dicts = IAA.df_overlaps(john_df_doc1,mengke_df_doc1,labels=0)
dicts

({30: [21, 18],
  31: [18, 22],
  32: [19, 23],
  33: [19, 24],
  34: [19, 25],
  35: [27],
  36: [28],
  37: [31],
  38: [32],
  39: [33],
  40: [34],
  41: [20, 35],
  42: [20, 36],
  43: [20, 37],
  44: [20, 38],
  45: [20, 39],
  46: [20, 40],
  47: [20, 41],
  69: [42],
  75: [43],
  78: [44],
  84: [28,
   19,
   23,
   24,
   25,
   26,
   27,
   20,
   34,
   29,
   30,
   31,
   32,
   33,
   35,
   36,
   37,
   39,
   38,
   40,
   41],
  87: [42],
  88: [44, 43, 45]},
 {21: [30],
  18: [30, 31],
  22: [31],
  19: [32, 33, 34, 84],
  23: [32, 84],
  24: [33, 84],
  25: [34, 84],
  27: [35, 84],
  28: [36, 84],
  31: [37, 84],
  32: [38, 84],
  33: [39, 84],
  34: [40, 84],
  20: [41, 42, 43, 44, 45, 46, 47, 84],
  35: [41, 84],
  36: [42, 84],
  37: [43, 84],
  38: [44, 84],
  39: [45, 84],
  40: [46, 84],
  41: [47, 84],
  42: [69, 87],
  43: [75, 88],
  44: [78, 88],
  26: [84],
  29: [84],
  30: [84],
  45: [88]})

In [7]:
IAA.create_agreement_df(dicts[0],dicts[1],john_df_doc1,mengke_df_doc1)

5585.txt
30
31
32
33


IndexError: list index out of range

In [8]:
test_dict = {"Test1" : [4,5,6]}
test_dict["Test1"][0] += 2
test_dict

{'Test1': [6, 5, 6]}

In [ ]:
##This was a very bad idea

def match_indices_recursive(doc1_matches,doc2_matches,index,doc1_doc2,doc1_indices,doc2_indices):  
    if (doc1_doc2==1):
        doc1_indices.add(index)
        for index2 in doc1_matches[index]:
            if index2 not in doc2_indices():
                doc2_indices.add(index2)
                doc2_indices.add(match_indices_recursive(doc1_matches,doc2_matches,index2,2,doc1_indices,doc2_indices))
    elif doc1_doc2==2:
        doc2_indices.add(index)
        for index1 in doc2_matches[index]:
            return (match_indices_recursive(doc1_matches,doc2_matches,index2,1,doc1_indices,doc2_indices)[0].add(index1))
    else:
        return [set(),set()]

In [34]:
test = pd.DataFrame(columns=['test','test2'])
test2 = pd.DataFrame({'test': [2,3], 'test2' : [3,4]})
pd.concat([test,test2])

,test,test2
0,2,3
1,3,4


In [5]:
def create_agreement_df(doc1_matches,doc2_matches,doc1_ents,doc2_ents):
    result_dict = {"Index" : [],"Annotation_1" : [],"Annotation_2" : [], "Exact Match?" : [], "Duplicate Matches?" : [], "Overlap?" : []} 
    
    #Add doc name,labels,start_char(1&2),end_char(1&2), fix index, get rid of index column
    
    for index1 in range(doc1_ents.shape[0]): #iterate through all ents inset one
        if index1 in doc1_matches.keys(): #if ent is in
            #if another index1 is in doc2_matches.values(), then add it to this row
            first_index2 = sorted(doc1_matches[index1])[0]
            first_index1 = sorted(doc2_matches[first_index2])[0]
            if first_index1 < index1:
                #Add to index: sorted(doc2_matches[first_index2])[0]
                duplicate_match_index = result_dict["Index"].index(first_index1)
                result_dict["Index"][duplicate_match_index] += " || " + doc1_ents.loc[index1,'Span Text']
                result_dict["Duplicate Matches?"][duplicate_match_index] = 1
            else:
                result_dict["Index"].append(index1)
                result_dict["Annotation_1"].append(doc1_ents.loc[index1,'Span Text'])
                annot_2 = ""
                first_time=1
                for index2 in sorted(doc1_matches[index1]):
                    if first_time ==1:
                        annot_2 += doc2_ents.loc[index2,'Span Text']
                        first_time=0
                    else:
                        annot_2 += " || " + doc2_ents.loc[index2,'Span Text']
                result_dict["Annotation_2"].append(annot_2)
                result_dict["Exact Match?"].append("")
                if len(doc1_matches[index1]) > 1:
                    result_dict["Duplicate Matches?"].append(1)
                else:
                    result_dict["Duplicate Matches?"].append(0)
                result_dict["Overlap?"].append(1)
        else:
            result_dict["Index"].append(index1)
            result_dict["Annotation_1"].append(doc1_ents.loc[index1,'Span Text'])
            result_dict["Annotation_2"].append("")
            result_dict["Exact Match?"].append(0)
            result_dict["Duplicate Matches?"].append(0)
            result_dict["Overlap?"].append(0)
    for index2 in range(doc2_ents.shape[0]):
        if index2 not in doc2_matches.keys():
            result_dict["Index"].append(index2)
            result_dict["Annotation_1"].append("")
            result_dict["Annotation_2"].append(doc1_ents.loc[index2,'Span Text'])
            result_dict["Exact Match?"].append(0)
            result_dict["Duplicate Matches?"].append(0)
            result_dict["Overlap?"].append(0)
        #Add annotation2
    return pd.DataFrame.from_dict(result_dict)